In [ ]:
import numpy as np
import open3d as o3d
import multiprocessing as mp

def worker(theta):
    theta = np.deg2rad(theta)
    f = robot.forward(theta)
    # Return end frame position
    return f.t_3_1.reshape([3, ])

def main():
    np.set_printoptions(precision=3, suppress=True)
    dh_params = np.array([[0.25, 0, np.pi/2, 0],
                          [0, 0.25, 0, 0],
                          [0, 0.25, 0, 0],
                          [0, 0.1, 0, 0],
                          [0, 0, -np.pi/2, np.pi/2]])

    global robot
    robot = RobotSerial(dh_params)

    # Create range of angles based on joint limitations
    angle_range_q1 = np.arange(-80, 80, 30)
    angle_range_q2_q5 = np.arange(-90, 90, 30)

    thetas = []
    for a in angle_range_q1:
        for b in angle_range_q2_q5:
            for c in angle_range_q2_q5:
                for d in angle_range_q2_q5:
                    for e in angle_range_q2_q5:
                        thetas.append(np.array([a, b, c, d, e]))

    pool = mp.Pool(mp.cpu_count())
    positions = pool.map(worker, thetas)
    pool.close()

    positions = np.array(positions)  # Convert list to numpy array

    # Create open3d point cloud object
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(positions)

    # Save the point cloud to a file
    o3d.io.write_point_cloud("points.pcd", pcd)

    # Visualize the point cloud
    o3d.visualization.draw_geometries([pcd])

    robot.show()

if __name__ == "__main__":
    main()


In [11]:
import open3d as o3d

# Load point cloud from file
pcd = o3d.io.read_point_cloud("points.pcd")

# Create a coordinate frame
frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5, origin=[0, 0, 0])

# Create a visualizer
vis = o3d.visualization.Visualizer()

# Add the point cloud and the coordinate frame
vis.create_window(width=800, height=800)
vis.add_geometry(pcd)
vis.add_geometry(frame)

# Run the visualizer
vis.run()
vis.capture_screen_image("my_image2.png")
# Close the visualizer
vis.destroy_window()
